In [15]:
import importlib
import helper
importlib.reload(helper)
%matplotlib inline

import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import mean_squared_error as mse

from skimage.metrics import structural_similarity as ssim
from helper import Timer, remove_dupe_frames

import ray
import time

In [21]:
ray.shutdown()
# ray.init()

In [18]:
# Testing ray
@ray.remote
def f(x):
    time.sleep(1)
    return x

result_ids = []
timer.start()
for i in range(10):
    result_ids.append(f.remote(i))
results = ray.get(result_ids)
timer.stop()
print(f"Time elapsed with ray: {timer.elapsedTime()} for result = {results}")

def f2(x):
    time.sleep(1)
    return x

result = []
timer.start()
for i in range(10):
    time.sleep(1)
    result.append(f2(i))
timer.stop()
print(f"Time elapsed without ray: {timer.elapsedTime()} for result = {result}")


Time elapsed with ray: 5.04 for result = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Time elapsed without ray: 20.07 for result = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [19]:
timer = Timer()
images = []
gs_frames = []
capture = cv2.VideoCapture('sampleClips/video.mp4')

@ray.remote
def get_gray_scale(frame):
    return cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

timer.start()
futures = []
while True:
    success, frame = capture.read()
    if not success:
        gray_frame = None
        break

    images.append(frame)
    # TODO: future implementations for frame processing
    futures.append(get_gray_scale.remote(frame))

gs_frames = ray.get(futures)

timer.stop()
print(f"Splitting frames: Elapsed time {timer.elapsedTime()}")

(raylet) Spilled 2479 MiB, 424 objects, write throughput 211 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.
(raylet) Spilled 4180 MiB, 531 objects, write throughput 194 MiB/s.
(raylet) Spilled 8301 MiB, 792 objects, write throughput 174 MiB/s.


Splitting frames: Elapsed time 123.80


Elapsed time with single thread task: 5.18, 3,47, 2.6, 2.51, 3.39
Elapsed time with parallel tasks: 6.48, 7.5, 8.1,

In [ ]:
print(f"Frame count: {len(images)}")
img = images[0]
img.shape, img.size, img.dtype

In [ ]:
gs_img = gs_frames[0]
gs_img.shape, gs_img.size, gs_img.dtype

In [ ]:
timer.stop()
unique_frames = remove_dupe_frames(gs_frames, timer)

In [ ]:
print(len(unique_frames))

In [ ]:
rows, cols = round(len(unique_frames)/3), 3
fig, axes = plt.subplots(rows, cols, figsize=(5 * cols,5 * rows))
len(axes)

In [ ]:
rows, cols = axes.shape # (2,3)
for row in range(rows):
    for col in range(cols):
        if row * cols + col + 1 > len(unique_frames):
            # hide axis if its not supposed to be plotted
            axes[row][col].set_axis_off()
            break
        axes[row][col].imshow(unique_frames[row * cols + col], cmap=plt.cm.gray)
        axes[row][col].set_title(f"Frame {row * cols + col + 1}")

In [ ]:
# adjust padding btwn subplots
plt.tight_layout()
# show plot
plt.show()